In [1]:
import pandas as pd
import random
import os
import gc

import matplotlib.pyplot as plt

import numpy as np

from bokeh.palettes import Spectral10
from bokeh.io import output_notebook, show

output_notebook()
                                                                                                                                                                                                                                                                                                                                                        
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

palette = Spectral10

pd.set_option('display.float_format', lambda x: '%.3f' % x)

import bokeh
bokeh.__version__

os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())

from src.dao.csv_dao import load_user_gps_csv, load_user_gps_time_window, load_user_stop_regions_centroids
from src.dao.dbdao import DBDAO
from src.utils.math import normalize
from src.utils.time_utils import enrich_time_columns, local_time
from src.utils.geo import cluster_centroid, weighted_cluster_centroid
from src.data_processment.stop_region import MovingCentroidStopRegionFinder
from src.plot.plot import add_centroid_figure, add_calculated_centroid_figure
from src.utils.geo import index_clusters
from src.plot.plot import plot_user_loc, plot_stop_regions_centroids
from bokeh.plotting import gridplot

from src.dao import places_dao

Loading BokehJS ...

working dir /home/tales/dev/master/mdc_analysis


In [2]:
def load_users_gps_data(userids, cols=["userid", "latitude", "longitude", "tz", "time", "local_time", "horizontal_accuracy", "horizontal_dop", "speed"]):
    df = pd.DataFrame()
    for userid in userids:
        user_gps_df = load_user_gps_csv(userid)
        user_gps_df["userid"] = [userid] * len(user_gps_df)
        df = df.append(user_gps_df)
        
    df = local_time(df)
    
    if cols != "*":
        df = df[cols]
    
    df = df.sort_values("local_time")

    return df

def load_user_gps_data(userid, cols=["userid", "latitude", "longitude", "tz", "time", "local_time", "horizontal_accuracy", "horizontal_dop", "speed"]):
    user_gps_data = load_user_gps_csv(userid)
    user_gps_data["userid"] = [userid] * len(user_gps_data)
    
    if cols != "*":
        user_gps_data = user_gps_data[cols]
        
    user_gps_data = user_gps_data.sort_values("local_time")
    
    return user_gps_data

In [3]:
user_ids = DBDAO().users_with_places().sample(15).tolist()
user_ids

[6026,
 6063,
 5925,
 5973,
 5988,
 5928,
 6168,
 6028,
 6087,
 6030,
 5949,
 5960,
 5979,
 6038,
 5962]

In [4]:
from src.plot.plot import plot_user_loc
from bokeh.plotting import gridplot

def plot_visit(visit_locations, color="navy", title="user visits", limit_points=500, p=None, legend=None, width=800, height=600):
    if len(visit_locations) > limit_points:
        visit_locations = visit_locations.sample(limit_points)
    p = plot_user_loc(visit_locations[["latitude", "longitude"]].drop_duplicates(), title=title, width=width, height=height, size=4, color=color, p=p, legend=legend)
    return p

# Categoria WORK

### Centróide da categoria WORK

In [5]:
def remove_outliers(data, quantile_threshold=0.05):
    clean_data = data.copy()
    
    if len(clean_data) == 0:
        return pd.DataFrame()
    
    lower_lat_value = data["latitude"].quantile(quantile_threshold)
    higher_lat_value = data["latitude"].quantile(1 - quantile_threshold)
    lower_lon_value = data["longitude"].quantile(quantile_threshold)
    higher_lon_value = data["longitude"].quantile(1 - quantile_threshold)
    
    clean_data = clean_data[(clean_data["latitude"] >= lower_lat_value) & (clean_data["latitude"] <= higher_lat_value) & (clean_data["longitude"] >= lower_lon_value) & (clean_data["longitude"] <= higher_lon_value)]
    
    return clean_data

In [6]:
for user_id in user_ids:
    work_data = places_dao.places_work(user_id)
    print(user_id, "--", len(work_data))
    
    if len(work_data) == 0:
        print("User has no point associated to workplace")
        continue
    
    work_data["w_acc_complement"] = normalize(work_data["horizontal_accuracy"], complement=True)
    work_data["w_dop_complement"] = normalize(work_data["horizontal_dop"], complement=True)    
    
    p_work = plot_visit(work_data, color="navy", limit_points=10000, title=str(user_id) + " WORK place", legend="visits")    
    
    if len(work_data) > 1:
        add_centroid_figure(p_work, cluster=work_data, point_color="navy", fill_color="navy", cluster_alpha=0.3)
        add_calculated_centroid_figure(p_work, centroid=weighted_cluster_centroid(work_data, "w_acc_complement"), legend="visits W Ac", point_color="green", fill_color="green", cluster_alpha=0.3)
        add_calculated_centroid_figure(p_work, centroid=weighted_cluster_centroid(work_data, "w_dop_complement"), legend="visits W DOP", point_color="lightblue", fill_color="lightblue", cluster_alpha=0.3)

    stop_region_centroids = load_user_stop_regions_centroids(user_id)
    #print(stop_region_centroids)
    #add_stop_regions(p_work, stop_region_centroids, cluster_alpha=0.1)
        
    show(p_work)

6026 -- 1


6063 -- 1339


5925 -- 2690


5973 -- 708


5988 -- 273


...

6030 -- 0
User has no point associated to workplace
5949 -- 2606


5960 -- 1071


5979 -- 1445


6038 -- 193


5962 -- 1096


In [7]:
def stop_regions_work_deprecated(work_points, stop_regions):
    work_stop_regions = []
    
    for index_sr, sr in stop_regions.iterrows():
        stop_region_checked = False
        
        for index_hp, work_point in work_points.iterrows():
            if (not stop_region_checked) & (work_point["local_time"] > sr["local_start_time"]) & (work_point["local_time"] < sr["local_end_time"]):
                work_stop_regions.append(sr["sr_id"])
                stop_region_checked = True
                
    return work_stop_regions

def stop_regions_work(work_points, stop_regions):
    work_stop_regions = []
    
    for index_sr, sr in stop_regions.iterrows():
        stop_region_checked = False
        
        points_match = work_points[(work_points["local_time"] > sr["local_start_time"]) & (work_points["local_time"] < sr["local_end_time"])]
        
        if len(points_match) > 0:
            work_stop_regions.append(sr["sr_id"])
                        
    return work_stop_regions

def plot_points_and_centroids(points, centroids, user_id):
    p = plot_stop_regions_centroids(centroids, title="User: {} - GPS points and Stop Region".format(str(user_id)))
    p = plot_user_loc(user_data=points, width=600, height=400, p=p)
    show(p)

In [8]:
user_id = 6085
user_sr = load_user_stop_regions_centroids(user_id)
print("{} stop regions".format(len(user_sr)))
work_points = remove_outliers(places_dao.places_work(user_id))

sr_ids = stop_regions_work(work_points, user_sr)
print("{} stop regions on HOME".format(len(sr_ids)))

plot_points_and_centroids(work_points, user_sr[user_sr["sr_id"].isin(sr_ids)], user_id)

1398 stop regions
68 stop regions on HOME


In [14]:
for user_id in DBDAO().users_with_places():
    print()
    print("user_id: {}".format(user_id))
    user_sr = load_user_stop_regions_centroids(user_id)
    print("{} stop regions".format(len(user_sr)))
    
    dirpath = "outputs/work_inferred/"
    filename = "work_stop_regions_user_{}.csv".format(user_id)
    
    if os.path.exists(dirpath + filename):
        print("User already computed")
        continue
    
    work_points = remove_outliers(places_dao.places_work(user_id))
    
    if len(work_points) == 0:
        print("User has no points within working time")
        continue
    
    sr_ids = stop_regions_work(work_points, user_sr)
    print("{} stop regions on WORK".format(len(sr_ids)))
    
    pd.Series(sr_ids).rename("sr_id").to_csv(dirpath + filename,
                                             header=True, index=False)


user_id: 5924
628 stop regions
99 stop regions on WORK

user_id: 5925
2659 stop regions
243 stop regions on WORK

user_id: 5927
3321 stop regions
User has no points within working time

user_id: 5928
4546 stop regions
44 stop regions on WORK

user_id: 5935
372 stop regions
6 stop regions on WORK

user_id: 5936
761 stop regions
62 stop regions on WORK

user_id: 5937
118 stop regions
11 stop regions on WORK

user_id: 5938
6047 stop regions
680 stop regions on WORK

user_id: 5940
757 stop regions
18 stop regions on WORK

user_id: 5942
956 stop regions
216 stop regions on WORK

user_id: 5943
251 stop regions
18 stop regions on WORK

user_id: 5945
1193 stop regions
190 stop regions on WORK

user_id: 5947
2029 stop regions
156 stop regions on WORK

user_id: 5948
996 stop regions
83 stop regions on WORK

user_id: 5949
930 stop regions
60 stop regions on WORK

user_id: 5950
649 stop regions
13 stop regions on WORK

user_id: 5953
1443 stop regions
55 stop regions on WORK

user_id: 5956
308 sto